<font size="20">

# LET measurements

<font size="3">
    
> __Author:__ XXX
<br/>__Institute:__ XXX
<br/>__Date:__ XX/XX-XXXX

_Abstract:_
This notebook contains the data-analysis tools used for the measurements at the LET instrument at ISIS. It describes how the analysis was performed, and summarizes the results.

# Contents
    
* [1. Introduction](#INTRODUCTION)
    * [1.1 Packages](#PACKAGES)
    * [1.2 Parameters](#PARAMETERS)
    * [1.3 Functions](#FUNCTIONS)
* [2. Data](#DATA)
    * [2.1 Paths](#PATHS)
    * [2.2 Extract](#EXTRACT)
    * [2.3 Load](#LOAD)
* [3. Filters](#FILTER)
* [4. Plotting](#PLOTTING)

# 1. Introduction

...

## 1.1 Packages<a class="anchor" id="PACKAGES"></a>

Import all necessary packages.

In [ ]:
# Autoload packages when doing an external change
%load_ext autoreload
%autoreload 2

# Activate matplotlib in interactive notebook mode
%matplotlib

# General packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import plotly.graph_objects as go
from plotly.offline import iplot, plot

# Data handling
import mg.file_handling.read as mg_read

# Plotting
import mg_let.basic_plot as mg_basic_plot

# Helper functions
import mg.helper_functions.misc as mg_hf

## 1.2 Parameters

In [ ]:
# Declare all global parameters
RAW_FOLDER = '../data/template/raw/'
PROCESSED_FOLDER = '../data/template/processed/'

## 1.3 Global functions

Declare all functions used in the analysis.

### Extract

In [ ]:
def extract_mg_data(zipped_path, clusters_save_path, events_save_path):
    """
    Function to extract, cluster and save data.
    
    Args:
        zipped_path (str): Location of raw data
        clusters_save_path (str): Destination for clusters
        events_save_path (str): Destination for events

    Yields:
        Clusters and events are extracted from the raw data and saved at the specified locations
    
    """
    unzipped_path = mg_read.unzip_data(zipped_path)
    data = mg_read.import_data(unzipped_path)
    # Extract clusters and save to disc
    clusters = mg_read.extract_clusters(data)
    mg_read.save_data(clusters, clusters_save_path)
    clusters = None
    # Extract events and save to disc
    events = mg_read.extract_events(data)
    mg_read.save_data(events, events_save_path)
    events = None
    # Clear data
    data = None

In [ ]:
# Extract data, cluster, and save to file
def extract_and_save(run, raw_path):
    """
    Function to extract, cluster and save data.
    
    Args:
        run (str): File run, as specified in the 'Paths'-declaration below
        raw_path (str): Path to the raw data in the '.zip'-file

    Yields:
        Clusters and events are extracted from the raw data and saved in the 'processed'-folder.
    
    """
    clusters_path = PROCESSED_FOLDER + run + '_clu.h5'
    events_path = PROCESSED_FOLDER + run + '_ev.h5'
    extract_mg_data(raw_path, clusters_path, events_path)

### Load

In [ ]:
# Load clusters and events
def load_clusters_and_events(run):
    """
    Function to load data from a specific run.
    
    Args:
        run (str): File run, as specified in the 'Paths'-declaration below

    Returns:
        Clusters (DataFrame)
        Events (DataFrame)
    
    """
    clusters_path = PROCESSED_FOLDER + run + '_clu.h5'
    events_path = PROCESSED_FOLDER + run + '_ev.h5'
    return mg_read.load_data(clusters_path), mg_read.load_data(events_path)

### Plotting

In [ ]:
def plot_basic(run, clusters_unfiltered, events, df_filter, bus_start, bus_stop, number_rows, area):
    """
    Function to plot all basic plots for SEQUOIA detector,
    such as PHS, Coincidences and rate.
    
    Ordering of plotting is:
    
    PHS 2D - NOT FILTERED
    MULTIPLICITY - FILTERED
    PHS 1D - FILTERED
    COINCIDENCES 2D - FILTERED
    PHS CORRELATION - FILTERED
    RATE - FILTERED
    TOF - FILTERED
    
    Note that all plots are filtered except the PHS 2D.
    
    Args:
        run (str): File run
        clusters_unfiltered (DataFrame): Unfiltered clusteres
        events (DataFrame): Individual events
        df_filter (dict): Dictionary specifying the filter which will be used on the clustered data
        bus_start (int): First bus to plot
        bus_stop (int): Last bus to plot
        number_rows (int): Number of rows in plots (number of rows must be larger than number_buses/3)
        area (float): Area in m^2 of the active detector surface

    Yields:
        Plots the basic analysis
    
    """
    mg_hf.set_thick_labels(12)
    
    # Filter clusters
    clusters = mg_read.filter_data(clusters_unfiltered, df_filter)
    
    # Declare parameters
    duration = (clusters_unfiltered.time.values[-1] - clusters_unfiltered.time.values[0]) * 62.5e-9
    
    # PHS - 2D
    vmin = 1
    vmax = events.shape[0] // 1000 + 100
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        events_bus = events[events.bus == bus]
        if events_bus.shape[0] > 0:
            mg_basic_plot.phs_2d_plot(events_bus, bus, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_phs_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Multiplicity
    vmin = None
    vmax = None
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 1:
            mg_basic_plot.multiplicity_plot_perc(clusters_bus, bus, duration)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_multiplicity_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # PHS - 1D
    vmin = None
    vmax = None
    bins_phs_1d = 300
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        clusters_uf_bus = clusters_unfiltered[clusters_unfiltered.bus == bus]
        mg_basic_plot.phs_clusters_1d_plot(clusters_bus, clusters_uf_bus, bins_phs_1d, bus, duration)
        plt.yscale('log')
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_phs_1d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - 2D
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    if clusters.shape[0] != 0:
        vmin = (1 * 1/duration)
        vmax = (clusters.shape[0] // 450 + 5) * 1/duration
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        # Calculate number of events and rate in a specific bus
        number_events = clusters_bus.shape[0]
        events_per_s = number_events/duration
        events_per_s_m2 = events_per_s/area
        title = ('Bus %d\n(%d events, %.6f events/s/m$^2$)' % (bus, number_events, events_per_s_m2))
        if number_events > 1:
            mg_basic_plot.clusters_2d_plot(clusters_bus, title, vmin, vmax, duration)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_coincidences_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - PHS
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    if clusters.shape[0] != 0:
        vmin = 1/duration
        vmax = (clusters.shape[0] // 450 + 1000) / duration
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 1:
            mg_basic_plot.clusters_phs_plot(clusters_bus, bus, duration, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_coincidences_phs.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Rate 
    number_bins = 50
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.rate_plot(clusters_bus, number_bins, bus, area)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_rate.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    
    # TIME-OF-FLIGHT
    number_bins = 300
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.tof_histogram(clusters_bus, number_bins, bus)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_tof.png' % run
    fig.savefig(output_path, bbox_inches='tight')

# 2. Data

## 2.1 Paths<a class="anchor" id="EXTRACT"></a>

In [ ]:
PATHS = {'run_1': RAW_FOLDER + 'path_1.zip',
         'run_2': RAW_FOLDER + 'path_2.zip',
         'run_3': RAW_FOLDER + 'path_3.zip'}

## 2.2 Extract<a class="anchor" id="EXTRACT"></a>

In [ ]:
# Note that the 'extract_and_save'-function only needs to performed once.
# After that, the data has been clustered and saved. The data can then
# be accessed using the 'load_clusters_and_events'-function.

#extract_and_save('run_1', SEQUOIA_PATHS['run_1'])
#extract_and_save('run_2', SEQUOIA_PATHS['run_2'])
extract_and_save('run_3', SEQUOIA_PATHS['run_3'])

## 2.3 Load<a class="anchor" id="LOAD"></a>

In [ ]:
clu_run_1, ev_run_1 = load_clusters_and_events('run_1')
clu_run_2, ev_run_2 = load_clusters_and_events('run_2')
clu_run_3, ev_run_3 = load_clusters_and_events('run_3')

# 3. Filters<a class="anchor" id="FILTER"></a>

Declare filters which can be used on the data. Below are a few template filters.

## SEQUOIA

In [ ]:
"""
Filters are declared in the following format: {'PARAMETER': [MIN_VALUE, MAX_VALUE, IS_ACTIVATE]}
"""

# Declare filters for SEQUOIA module
mg_filter_basic = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [600, np.inf, True],         # Wire charge
                   'gadc': [600, np.inf, True],         # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, True]}               # Grid channel, bottom=80 to top=119          

# 4. Plotting<a class="anchor" id="FILTER"></a>

### run_1 (data_set_name_1)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('sequoia_run_1', clu_sequoia_run_1, ev_sequoia_run_1, mg_filter_basic, 0, 5, 2, area)

### run_2 (data_set_name_2)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('sequoia_run_3', clu_sequoia_run_3, ev_sequoia_run_3, mg_filter_no_edges_no_back, 0, 5, 2, area)